Importing the packages used:

In [29]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

Loading in the hitters by game CSV

In [2]:
hitters = pd.read_csv('/mnt/g/ML_data/BB_game_data/hittersByGame.csv')

/tmp/ipykernel_113/2452900381.py:1: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  hitters = pd.read_csv('/mnt/g/ML_data/BB_game_data/hittersByGame.csv')


Listing the entire CSV out

In [3]:
hitters

,Hitters,H-AB,AB,R,H,RBI,BB,K,#P,AVG,OBP,SLG,Game,Team,Position,Hitter Id
0,M. Carpenter,1-4,4,0,1,1,1,0,22,0.25,0.4,0.25,360403123,STL,3B,31015
1,T. Pham,0-1,1,0,0,0,0,0,1,0.0,0.0,0.0,360403123,STL,LF,31208
2,M. Adams,0-4,4,0,0,0,0,2,18,0.0,0.0,0.0,360403123,STL,1B,31359
3,M. Holliday,1-3,3,0,1,0,1,1,15,0.33299999999999996,0.5,0.33299999999999996,360403123,STL,1B-LF,5940
4,R. Grichuk,1-4,4,0,1,0,0,3,14,0.25,0.25,0.25,360403123,STL,CF,31399
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361859,A. Diaz,0-1,1,0,0,0,0,1,5,0.333,0.5,0.333,401361981,HOU,PH-LF,33390
361860,M. Maldonado,1-2,2,0,1,0,0,1,12,0.235,0.263,0.235,401361981,HOU,C,30289
361861,M. Gonzalez,0-1,1,0,0,0,0,1,6,0.25,0.25,0.25,401361981,HOU,PH,30327
361862,G. Stubbs,0-0,0,0,0,0,0,0,0,0.0,0.0,0.0,401361981,HOU,C,35760


Here I am removing out all of the values that are not intergers. No model can accept strings.

In [4]:
hitter_names_ids = hitters[["Hitters","Hitter Id"]].copy
col_to_remove = ['Game','Team','Position','H-AB',"Hitters","Hitter Id"]
ml_hitters = hitters.drop(columns=col_to_remove, axis=1, inplace=True)
hitters.drop(hitters.index[-1],inplace=True)
hitters = hitters.apply(pd.to_numeric, errors='coerce')
hitters = hitters.dropna()

Listing out the Data Frame

In [5]:
hitters

,AB,R,H,RBI,BB,K,#P,AVG,OBP,SLG
0,4.0,0.0,1.0,1.0,1.0,0.0,22.0,0.250,0.400,0.250
1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000,0.000,0.000
2,4.0,0.0,0.0,0.0,0.0,2.0,18.0,0.000,0.000,0.000
3,3.0,0.0,1.0,0.0,1.0,1.0,15.0,0.333,0.500,0.333
4,4.0,0.0,1.0,0.0,0.0,3.0,14.0,0.250,0.250,0.250
...,...,...,...,...,...,...,...,...,...,...
361858,2.0,0.0,0.0,0.0,0.0,0.0,4.0,0.167,0.167,0.167
361859,1.0,0.0,0.0,0.0,0.0,1.0,5.0,0.333,0.500,0.333
361860,2.0,0.0,1.0,0.0,0.0,1.0,12.0,0.235,0.263,0.235
361861,1.0,0.0,0.0,0.0,0.0,1.0,6.0,0.250,0.250,0.250


Now I am seeting up the Linear regression model.

In [6]:
y = hitters["OBP"]
x = hitters.copy()
x = x.drop("OBP", axis=1)

#removing any nan values
x.isna().any().any()

In [12]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [13]:
# Create an instance of the LinearRegression model
model = LinearRegression()

# Fit the model to the training data
model.fit(X_train, y_train)

# Predict the target variable for new data
y_pred = model.predict(X_test)

In [14]:
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(rmse)

In [17]:
mse = mean_squared_error(y_test, y_pred)
print(mse)

In [27]:
coefficients = model.coef_
indices = np.argsort(coefficients)[::-1]
for i in range(X_train.shape[1]):
    print(f"{i + 1}. Feature '{X_train.columns[indices[i]]}': {coefficients[indices[i]]}")


1. Feature 'AVG': 0.947071239916634
2. Feature 'SLG': 0.04469731686837419
3. Feature 'BB': 0.02297589195730991
4. Feature 'AB': 0.002577987776112637
5. Feature 'K': 0.0010316565374475048
6. Feature '#P': 0.0007574198583634676
7. Feature 'R': 0.00023867400809855083
8. Feature 'RBI': -0.00021992111543974076
9. Feature 'H': -0.0032577343867289097


This is crazy to think about. The AVG is one of the most important features for prediciting OBP.